In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
trainloc = '/content/gdrive/MyDrive/Data/Mask_Dataset/Train'
testloc = '/content/gdrive/MyDrive/Data/Mask_Dataset/Validation'
validationloc = '/content/gdrive/MyDrive/Data/Mask_Dataset/Validation'

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    trainloc,
    target_size=(128,128),
    batch_size=20,
    class_mode='binary'
    )

validation_generator = test_datagen.flow_from_directory(
    validationloc,
    target_size=(128,128),
    batch_size=20,
    class_mode='binary'
    )

Found 4400 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(128,128,3))

80142336/80134624 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
model = models.Sequential([
              conv_base,
              layers.Flatten(),
              layers.Dense(256,activation='relu'),
              layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 22,122,049
Trainable params: 22,122,049
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
               optimizer=optimizers.RMSprop(lr=2e-5),
               metrics = ['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("CNN Project Model-{epoch:02d}.h5")

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[checkpoint_cb])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
100/100 [==============================] - 1345s 13s/step - loss: 0.0095 - acc: 0.9910 - val_loss: 8.1527 - val_acc: 0.5000
Epoch 2/5
100/100 [==============================] - 1051s 10s/step - loss: 4.6299e-08 - acc: 1.0000
Epoch 3/5
100/100 [==============================] - 1026s 10s/step - loss: 9.1745e-10 - acc: 1.0000
Epoch 4/5
100/100 [==============================] - 1021s 10s/step - loss: 3.5406e-10 - acc: 1.0000
Epoch 5/5
100/100 [==============================] - 1023s 10s/step - loss: 1.4570e-10 - acc: 1.0000


In [ ]:
test_generator = test_datagen.flow_from_directory(
    testloc,
    target_size=(128,128),
    batch_size=20,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.


In [ ]:
model.evaluate_generator(test_generator, steps=20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[12.694567680358887, 0.48750001192092896]